In [1]:
import torch
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random
import torch.nn.init

torch.manual_seed(777) # reproducibility

In [2]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
keep_prob = 0.7

In [3]:
# MNIST dataset
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = True)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [4]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True)

In [5]:
# nn layers
linear1 = torch.nn.Linear(784, 512, bias = True)
linear2 = torch.nn.Linear(512, 512, bias = True)
linear3 = torch.nn.Linear(512, 512, bias = True)
linear4 = torch.nn.Linear(512, 512, bias = True)
linear5 = torch.nn.Linear(512, 10, bias = True)

In [6]:
relu = torch.nn.ReLU()
# p is the probability of being dropped in PyTorch
dropout = torch.nn.Dropout(p = 1 - keep_prob)

In [7]:
# xavier initializer
torch.nn.init.xavier_uniform(linear1.weight)
torch.nn.init.xavier_uniform(linear2.weight)
torch.nn.init.xavier_uniform(linear3.weight)
torch.nn.init.xavier_uniform(linear4.weight)
torch.nn.init.xavier_uniform(linear5.weight)

<ipython-input-7-9619946836a4>:2: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(linear1.weight)
<ipython-input-7-9619946836a4>:3: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(linear2.weight)
<ipython-input-7-9619946836a4>:4: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(linear3.weight)
<ipython-input-7-9619946836a4>:5: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(linear4.weight)
<ipython-input-7-9619946836a4>:6: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(linear5.weight)


Parameter containing:
tensor([[-0.0565,  0.0423, -0.0155,  ...,  0.1012,  0.0459, -0.0191],
        [ 0.0772,  0.0452, -0.0638,  ...,  0.0476, -0.0638,  0.0528],
        [ 0.0311, -0.1023, -0.0701,  ...,  0.0412, -0.1004,  0.0738],
        ...,
        [ 0.0334,  0.0187, -0.1021,  ...,  0.0280, -0.0583, -0.1018],
        [-0.0506, -0.0939, -0.0467,  ..., -0.0554, -0.0325,  0.0640],
        [-0.0183, -0.0123,  0.1025,  ..., -0.0214,  0.0220, -0.0741]],
       requires_grad=True)

In [8]:
# model
model = torch.nn.Sequential(linear1, relu, dropout,
                            linear2, relu, dropout,
                            linear3, relu, dropout,
                            linear4, relu, dropout,
                            linear5)

In [9]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss() # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [10]:
# train my model
for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = len(mnist_train) // batch_size

  for i, (batch_xs, batch_ys) in enumerate(data_loader):
    # reshape input image into [batch_size by 784]
    X = Variable(batch_xs.view(-1, 28 * 28))
    Y = Variable(batch_ys) # label is not one-hot encoded

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
  
  print("[Epoch: {:>4}] cost = {:>.9}".format(epoch + 1, avg_cost.data))

print('Learning Finished')

[Epoch:    1] cost = 0.311190426
[Epoch:    2] cost = 0.14621383
[Epoch:    3] cost = 0.112888664
[Epoch:    4] cost = 0.0968321115
[Epoch:    5] cost = 0.083166644
[Epoch:    6] cost = 0.0736089349
[Epoch:    7] cost = 0.0712304413
[Epoch:    8] cost = 0.0604720414
[Epoch:    9] cost = 0.0616469346
[Epoch:   10] cost = 0.0572122484
[Epoch:   11] cost = 0.0528458282
[Epoch:   12] cost = 0.0507918559
[Epoch:   13] cost = 0.0449848436
[Epoch:   14] cost = 0.0465771705
[Epoch:   15] cost = 0.045319438
Learning Finished


In [11]:
# Test model and check accuracy
model.eval()  # set the model to evaluation mode (dropout=False)

Sequential(
  (0): Linear(in_features=784, out_features=512, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.30000000000000004, inplace=False)
  (3): Linear(in_features=512, out_features=512, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.30000000000000004, inplace=False)
  (6): Linear(in_features=512, out_features=512, bias=True)
  (7): ReLU()
  (8): Dropout(p=0.30000000000000004, inplace=False)
  (9): Linear(in_features=512, out_features=512, bias=True)
  (10): ReLU()
  (11): Dropout(p=0.30000000000000004, inplace=False)
  (12): Linear(in_features=512, out_features=10, bias=True)
)

In [12]:
X_test = Variable(mnist_test.test_data.view(-1, 28 * 28).float())
Y_test = Variable(mnist_test.test_labels)

/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [13]:
prediction = model(X_test)
correct_prediction = (torch.max(prediction.data, 1)[1] == Y_test.data)
accuracy = correct_prediction.float().mean()
print('Accuracy:', accuracy)

Accuracy: tensor(0.9830)


In [14]:
# Get one and predict
r = random.randint(0, len(mnist_test) - 1)
X_single_data = Variable(mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float())
Y_single_data = Variable(mnist_test.test_labels[r:r + 1])

In [15]:
print("Label: ", Y_single_data.data)
single_prediction = model(X_single_data)
print("Prediction: ", torch.max(single_prediction.data, 1)[1])

Label:  tensor([3])
Prediction:  tensor([3])
